## Semantic ranking using the azure.search.documents library in the Azure SDK for Python

https://learn.microsoft.com/en-us/azure/search/search-get-started-semantic?tabs=python

This notebook demonstrates a semantic configuration in a search index and the semanti query syntax for reranking search results.

1. Create Index
2. Add document
3. Search

## Install packages and set variables

In [1]:
# ! pip install azure-search-documents==11.6.0b1 --quiet
# ! pip install azure-identity --quiet
# ! pip install python-dotenv --quiet

In [1]:
import os

# Provide variables
search_endpoint: str = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
search_api_key: str = os.environ["AZURE_SEARCH_API_KEY"]
index_name: str = "hotels-quickstart"

search_endpoint

'https://ava-wora-ais-001.search.windows.net'

## Create an index

In [2]:
from azure.core.credentials import AzureKeyCredential

credential = AzureKeyCredential(search_api_key)

from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch
)

# Create a search schema
index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)

fields = [
        SimpleField(name="HotelId", type=SearchFieldDataType.String, key=True),
        SearchableField(name="HotelName", type=SearchFieldDataType.String, sortable=True),
        SearchableField(name="Description", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
        SearchableField(name="Description_fr", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
        SearchableField(name="Category", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="Tags", collection=True, type=SearchFieldDataType.String, facetable=True, filterable=True),
        
        SimpleField(name="ParkingIncluded", type=SearchFieldDataType.Boolean, facetable=True, filterable=True, sortable=True),
        SimpleField(name="LastRenovationDate", type=SearchFieldDataType.DateTimeOffset, facetable=True, filterable=True, sortable=True),
        SimpleField(name="Rating", type=SearchFieldDataType.Double, facetable=True, filterable=True, sortable=True),

        ComplexField(name="Address", fields=[
            SearchableField(name="StreetAddress", type=SearchFieldDataType.String),
            SearchableField(name="City", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="StateProvince", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="PostalCode", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="Country", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        ])
    ]

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="HotelName"),
        keywords_fields=[SemanticField(field_name="Category")],
        content_fields=[SemanticField(field_name="Description")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

semantic_settings = SemanticSearch(configurations=[semantic_config])
scoring_profiles = []
suggester = [{'name': 'sg', 'source_fields': ['Tags', 'Address/City', 'Address/Country']}]

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields, suggesters=suggester, scoring_profiles=scoring_profiles, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 hotels-quickstart created


In [14]:
index_client.get_index(name=index_name)

In [25]:
# index_client functions
# 'analyze_text', 'close', 
# 'create_alias', 'create_index', 'create_or_update_alias', 'create_or_update_index', 'create_or_update_synonym_map', 'create_synonym_map', 
# 'delete_alias', 'delete_index', 'delete_synonym_map', 
# 'get_alias', 'get_index', 'get_index_statistics', 'get_search_client', 'get_service_statistics', 'get_synonym_map', 'get_synonym_map_names', 'get_synonym_maps', 
# 'list_alias_names', 'list_aliases', 'list_index_names', 'list_indexes', 'send_request'

In [27]:
index_client.list_aliases()

<iterator object azure.core.paging.ItemPaged at 0x26e4732bb50>

In [19]:
index_client.list_index_names()

<iterator object azure.core.paging.ItemPaged at 0x26e463118d0>

## Add a documents payload

In [3]:
# Create a documents payload
documents = [
    {
    "@search.action": "upload",
    "HotelId": "1",
    "HotelName": "Secret Point Motel",
    "Description": "The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.",
    "Description_fr": "L'hôtel est idéalement situé sur la principale artère commerciale de la ville en plein cœur de New York. A quelques minutes se trouve la place du temps et le centre historique de la ville, ainsi que d'autres lieux d'intérêt qui font de New York l'une des villes les plus attractives et cosmopolites de l'Amérique.",
    "Category": "Boutique",
    "Tags": [ "pool", "air conditioning", "concierge" ],
    "ParkingIncluded": "false",
    "LastRenovationDate": "1970-01-18T00:00:00Z",
    "Rating": 3.60,
    "Address": {
        "StreetAddress": "677 5th Ave",
        "City": "New York",
        "StateProvince": "NY",
        "PostalCode": "10022",
        "Country": "USA"
        }
    },
    {
    "@search.action": "upload",
    "HotelId": "2",
    "HotelName": "Twin Dome Motel",
    "Description": "The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts.",
    "Description_fr": "L'hôtel est situé dans une place du XIXe siècle, qui a été agrandie et rénovée aux plus hautes normes architecturales pour créer un hôtel moderne, fonctionnel et de première classe dans lequel l'art et les éléments historiques uniques coexistent avec le confort le plus moderne.",
    "Category": "Boutique",
    "Tags": [ "pool", "free wifi", "concierge" ],
    "ParkingIncluded": "false",
    "LastRenovationDate": "1979-02-18T00:00:00Z",
    "Rating": 3.60,
    "Address": {
        "StreetAddress": "140 University Town Center Dr",
        "City": "Sarasota",
        "StateProvince": "FL",
        "PostalCode": "34243",
        "Country": "USA"
        }
    },
    {
    "@search.action": "upload",
    "HotelId": "3",
    "HotelName": "Triple Landscape Hotel",
    "Description": "The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services.",
    "Description_fr": "L'hôtel est situé dans une place du XIXe siècle, qui a été agrandie et rénovée aux plus hautes normes architecturales pour créer un hôtel moderne, fonctionnel et de première classe dans lequel l'art et les éléments historiques uniques coexistent avec le confort le plus moderne.",
    "Category": "Resort and Spa",
    "Tags": [ "air conditioning", "bar", "continental breakfast" ],
    "ParkingIncluded": "true",
    "LastRenovationDate": "2015-09-20T00:00:00Z",
    "Rating": 4.80,
    "Address": {
        "StreetAddress": "3393 Peachtree Rd",
        "City": "Atlanta",
        "StateProvince": "GA",
        "PostalCode": "30326",
        "Country": "USA"
        }
    },
    {
    "@search.action": "upload",
    "HotelId": "4",
    "HotelName": "Sublime Cliff Hotel",
    "Description": "Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace.",
    "Description_fr": "Le sublime Cliff Hotel est situé au coeur du centre historique de sublime dans un quartier extrêmement animé et vivant, à courte distance de marche des sites et monuments de la ville et est entouré par l'extraordinaire beauté des églises, des bâtiments, des commerces et Monuments. Sublime Cliff fait partie d'un Palace 1800 restauré avec amour.",
    "Category": "Boutique",
    "Tags": [ "concierge", "view", "24-hour front desk service" ],
    "ParkingIncluded": "true",
    "LastRenovationDate": "1960-02-06T00:00:00Z",
    "Rating": 4.60,
    "Address": {
        "StreetAddress": "7400 San Pedro Ave",
        "City": "San Antonio",
        "StateProvince": "TX",
        "PostalCode": "78216",
        "Country": "USA"
        }
    }
]

In [53]:
import pandas as pd

document_df = pd.json_normalize(documents)
document_df

,@search.action,HotelId,HotelName,Description,Description_fr,Category,Tags,ParkingIncluded,LastRenovationDate,Rating,Address.StreetAddress,Address.City,Address.StateProvince,Address.PostalCode,Address.Country
0,upload,1,Secret Point Motel,The hotel is ideally located on the main comme...,L'hôtel est idéalement situé sur la principale...,Boutique,"[pool, air conditioning, concierge]",false,1970-01-18T00:00:00Z,3.6,677 5th Ave,New York,NY,10022,USA
1,upload,2,Twin Dome Motel,The hotel is situated in a nineteenth century...,L'hôtel est situé dans une place du XIXe siècl...,Boutique,"[pool, free wifi, concierge]",false,1979-02-18T00:00:00Z,3.6,140 University Town Center Dr,Sarasota,FL,34243,USA
2,upload,3,Triple Landscape Hotel,The Hotel stands out for its gastronomic excel...,L'hôtel est situé dans une place du XIXe siècl...,Resort and Spa,"[air conditioning, bar, continental breakfast]",true,2015-09-20T00:00:00Z,4.8,3393 Peachtree Rd,Atlanta,GA,30326,USA
3,upload,4,Sublime Cliff Hotel,Sublime Cliff Hotel is located in the heart of...,Le sublime Cliff Hotel est situé au coeur du c...,Boutique,"[concierge, view, 24-hour front desk service]",true,1960-02-06T00:00:00Z,4.6,7400 San Pedro Ave,San Antonio,TX,78216,USA


## Upload documents
- search_client.upload_documents()

In [4]:
# Upload documents to the index
search_client = SearchClient(endpoint=search_endpoint, index_name=index_name, credential=credential)

try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)
    index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)

Upload of new document succeeded: True


In [ ]:
# search_client functions 
# 'autocomplete', 'close', 'delete_documents', 'get_document', 'get_document_count', 
# 'index_documents', 'merge_documents', 'merge_or_upload_documents', 'search', 
# 'send_request', 'suggest', 'upload_documents'

In [5]:
search_client

<SearchClient [endpoint='https://ava-wora-ais-001.search.windows.net', index='hotels-quickstart']>

In [23]:
search_client.get_document_count()

4

In [18]:
index_client.get_index_statistics(index_name=index_name)

{'document_count': 4, 'storage_size': 39995, 'vector_index_size': 0}

In [36]:
index = index_client.get_index(index_name)
field_names = [field.name for field in index.fields]
print(field_names)

# Print all fields in the index
for field in index.fields:
    print(f"Name: {field.name}, Type: {field.type}, Searchable: {field.searchable}, Filterable: {field.filterable}, Sortable: {field.sortable}, Facetable: {field.facetable}")

['HotelId', 'HotelName', 'Description', 'Description_fr', 'Category', 'Tags', 'ParkingIncluded', 'LastRenovationDate', 'Rating', 'Address']
Name: HotelId, Type: Edm.String, Searchable: False, Filterable: False, Sortable: False, Facetable: False
Name: HotelName, Type: Edm.String, Searchable: True, Filterable: False, Sortable: True, Facetable: False
Name: Description, Type: Edm.String, Searchable: True, Filterable: False, Sortable: False, Facetable: False
Name: Description_fr, Type: Edm.String, Searchable: True, Filterable: False, Sortable: False, Facetable: False
Name: Category, Type: Edm.String, Searchable: True, Filterable: True, Sortable: True, Facetable: True
Name: Tags, Type: Collection(Edm.String), Searchable: True, Filterable: True, Sortable: False, Facetable: True
Name: ParkingIncluded, Type: Edm.Boolean, Searchable: False, Filterable: True, Sortable: True, Facetable: True
Name: LastRenovationDate, Type: Edm.DateTimeOffset, Searchable: False, Filterable: True, Sortable: True, Fa

In [40]:
import pandas as pd
# Create a list of dictionaries
fields_data = [
    {
        "Name": field.name,
        "Type": field.type,
        "Searchable": field.searchable,
        "Filterable": field.filterable,
        "Sortable": field.sortable,
        "Facetable": field.facetable
    }
    for field in index.fields
]

# Convert the list of dictionaries to a DataFrame
fields_df = pd.DataFrame(fields_data)

# Display the DataFrame
fields_df

,Name,Type,Searchable,Filterable,Sortable,Facetable
0,HotelId,Edm.String,False,False,False,False
1,HotelName,Edm.String,True,False,True,False
2,Description,Edm.String,True,False,False,False
3,Description_fr,Edm.String,True,False,False,False
4,Category,Edm.String,True,True,True,True
5,Tags,Collection(Edm.String),True,True,False,True
6,ParkingIncluded,Edm.Boolean,False,True,True,True
7,LastRenovationDate,Edm.DateTimeOffset,False,True,True,True
8,Rating,Edm.Double,False,True,True,True
9,Address,Edm.ComplexType,None,None,None,None


- Type:
  - Description: Specifies the data type of the field. 
  - Common types include Edm.String, Edm.Int32, Edm.Double, Edm.Boolean, Edm.DateTimeOffset, Edm.GeographyPoint, and Collection(Edm.String).
- Searchable:
  - Description: Indicates whether the field can be used in full-text search queries.
- Filterable:
  - Description: Indicates whether the field can be used to filter results in queries.
- Sortable:
  - Description: Indicates whether the field can be used to sort results. If a field is sortable, you can use it in the orderby parameter of a query.
- Facetable:
  - Description: Indicates whether the field can be used to create facets. Facets are counts of documents that match a search query, broken down by a field value.

### Azure Cognitive Search: `search_client.search()` Parameters

- **search_text**: `str` | A full-text search query expression; use "*" or omit this parameter to match all documents.
- **include_total_count**: `bool` | A value that specifies whether to fetch the total count of results. Default is `false`. Setting this value to `true` may have a performance impact. Note that the count returned is an approximation.
- **facets**: `list[str]` | The list of facet expressions to apply to the search query. Each facet expression contains a field name, optionally followed by a comma-separated list of name:value pairs.
- **filter**: `str` | The OData `$filter` expression to apply to the search query.
- **highlight_fields**: `str` | The comma-separated list of field names to use for hit highlights. Only searchable fields can be used for hit highlighting.
- **highlight_post_tag**: `str` | A string tag that is appended to hit highlights. Must be set with `highlightPreTag`. Default is `</em>`.
- **highlight_pre_tag**: `str` | A string tag that is prepended to hit highlights. Must be set with `highlightPostTag`. Default is `<em>`.
- **minimum_coverage**: `float` | A number between 0 and 100 indicating the percentage of the index that must be covered by a search query in order for the query to be reported as a success. This parameter can be useful for ensuring search availability even for services with only one replica. The default is 100.
- **order_by**: `list[str]` | The list of OData `$orderby` expressions by which to sort the results. Each expression can be either a field name or a call to either the `geo.distance()` or the `search.score()` functions. Each expression can be followed by `asc` to indicate ascending, and `desc` to indicate descending. The default is ascending order. Ties will be broken by the match scores of documents. If no `OrderBy` is specified, the default sort order is descending by document match score. There can be at most 32 `$orderby` clauses.
- **query_type**: `str` or `~azure.search.documents.models.QueryType` | A value that specifies the syntax of the search query. The default is `simple`. Use `full` if your query uses the Lucene query syntax. Possible values include: `simple`, `full`, `semantic`.
- **scoring_parameters**: `list[str]` | The list of parameter values to be used in scoring functions (for example, `referencePointParameter`) using the format name-values. For example, if the scoring profile defines a function with a parameter called `mylocation` the parameter string would be `"mylocation--122.2,44.8"` (without the quotes).
- **scoring_profile**: `str` | The name of a scoring profile to evaluate match scores for matching documents in order to sort the results.
- **semantic_query**: `str` | Allows setting a separate search query that will be solely used for semantic reranking, semantic captions, and semantic answers. Useful for scenarios where there is a need to use different queries between the base retrieval and ranking phase, and the L2 semantic phase.
- **search_fields**: `list[str]` | The list of field names to which to scope the full-text search. When using fielded search (`fieldName:searchExpression`) in a full Lucene query, the field names of each fielded search expression take precedence over any field names listed in this parameter.
- **search_mode**: `str` or `~azure.search.documents.models.SearchMode` | A value that specifies whether any or all of the search terms must be matched in order to count the document as a match. Possible values include: `any`, `all`.
- **query_language**: `str` or `~azure.search.documents.models.QueryLanguage` | The language of the search query. Possible values include: `none`, `en-us`, `en-gb`, `en-in`, `en-ca`, `en-au`, `fr-fr`, `fr-ca`, `de-de`, `es-es`, `es-mx`, `zh-cn`, `zh-tw`, `pt-br`, `pt-pt`, `it-it`, `ja-jp`, `ko-kr`, `ru-ru`, `cs-cz`, `nl-be`, `nl-nl`, `hu-hu`, `pl-pl`, `sv-se`, `tr-tr`, `hi-in`, `ar-sa`, `ar-eg`, `ar-ma`, `ar-kw`, `ar-jo`, `da-dk`, `no-no`, `bg-bg`, `hr-hr`, `hr-ba`, `ms-my`, `ms-bn`, `sl-sl`, `ta-in`, `vi-vn`, `el-gr`, `ro-ro`, `is-is`, `id-id`, `th-th`, `lt-lt`, `uk-ua`, `lv-lv`, `et-ee`, `ca-es`, `fi-fi`, `sr-ba`, `sr-me`, `sr-rs`, `sk-sk`, `nb-no`, `hy-am`, `bn-in`, `eu-es`, `gl-es`, `gu-in`, `he-il`, `ga-ie`, `kn-in`, `ml-in`, `mr-in`, `fa-ae`, `pa-in`, `te-in`, `ur-pk`.
- **query_speller**: `str` or `~azure.search.documents.models.QuerySpellerType` | A value that specifies the type of the speller to use to spell-correct individual search query terms. Possible values include: `none`, `lexicon`.
- **query_answer**: `str` or `~azure.search.documents.models.QueryAnswerType` | This parameter is only valid if the query type is `semantic`. If set, the query returns answers extracted from key passages in the highest ranked documents. Possible values include: `none`, `extractive`.
- **query_answer_count**: `int` | This parameter is only valid if the query type is `semantic` and query answer is `extractive`. Configures the number of answers returned. Default count is 1.
- **query_answer_threshold**: `float` | This parameter is only valid if the query type is `semantic` and query answer is `extractive`. Configures the confidence threshold. Default count is 0.7.
- **query_caption**: `str` or `~azure.search.documents.models.QueryCaptionType` | This parameter is only valid if the query type is `semantic`. If set, the query returns captions extracted from key passages in the highest ranked documents. Defaults to `None`. Possible values include: `none`, `extractive`.
- **query_caption_highlight_enabled**: `bool` | This parameter is only valid if the query type is `semantic` when query caption is set to `extractive`. Determines whether highlighting is enabled. Defaults to `true`.
- **semantic_fields**: `list[str]` | The list of field names used for semantic search.
- **semantic_configuration_name**: `str` | The name of the semantic configuration that will be used when processing documents for queries of type `semantic`.
- **select**: `list[str]` | The list of fields to retrieve. If unspecified, all fields marked as retrievable in the schema are included.
- **skip**: `int` | The number of search results to skip. This value cannot be greater than 100,000. If you need to scan documents in sequence but cannot use `$skip` due to this limitation, consider using `$orderby` on a totally-ordered key and `$filter` with a range query instead.
- **top**: `int` | The number of search results to retrieve. This can be used in conjunction with `$skip` to implement client-side paging of search results. If results are truncated due to server-side paging, the response will include a continuation token that can be used to issue another `Search` request for the next page of results.
- **scoring_statistics**: `str` or `~azure.search.documents.models.ScoringStatistics` | A value that specifies whether we want to calculate scoring statistics (such as document frequency) globally for more consistent scoring, or locally, for lower latency. The default is `local`. Use `global` to aggregate scoring statistics globally before scoring. Using global scoring statistics can increase latency of search queries. Possible values include: `local`, `global`.
- **session_id**: `str` | A value to be used to create a sticky session, which can help get more consistent results. As long as the same `sessionId` is used, a best-effort attempt will be made to target the same replica set. Be wary that reusing the same `sessionId` values repeatedly can interfere with the load balancing of the requests across replicas and adversely affect the performance of the search service. The value used as `sessionId` cannot start with a `_` character.
- **semantic_error_mode**: `str` or `~azure.search.documents.models.SemanticErrorMode` | Allows the user to choose whether a semantic call should fail completely (default / current behavior), or to return partial results. Known values are: `partial` and `fail`.
- **semantic_max_wait_in_milliseconds**: `int` | Allows the user to set an upper bound on the amount of time it takes for semantic enrichment to finish processing before the request fails.
- **debug**: `str` or `~azure.search.documents.models.QueryDebugMode` | Enables a debugging tool that can be used to further explore your Semantic search results. Known values are: `disabled`, `speller`, `semantic`, and `all`.
- **vector_queries**: `list[VectorQuery]` | The query parameters for vector and hybrid search queries.
- **vector_filter_mode**: `str` or `VectorFilterMode` | Determines whether or not filters are applied before or after the vector search is performed. Default is `preFilter`. Known values are: `postFilter` and `preFilter`.


In [28]:
# Run an empty query (returns selected fields, all documents)
results =  search_client.search(
    query_type='simple',
    search_text="*" ,
    select='HotelName,Description',
    include_total_count=True
    )

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

Total Documents Matching Query: 4
1.0
Triple Landscape Hotel
Description: The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services.
1.0
Twin Dome Motel
Description: The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts.
1.0
Sublime Cliff Hotel
Description: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace.
1.0
Secret Point Motel
Description: The hotel is ideally located on the main commercial artery

In [68]:
## Search with filter and sort

# Execute the search query
results = search_client.search(
    search_text="first-class hotel",
    query_type="simple",
    search_fields=["HotelName", "Description", "Category", "Tags"],
    filter="Category eq 'Boutique' and Tags/any(t: t eq 'pool') and Rating gt 3 and ParkingIncluded eq false",
    order_by=["Rating desc"],
    facets=["Category", "Tags", "ParkingIncluded", "LastRenovationDate", "Rating"],
    top=3,
    skip=0,
    minimum_coverage=10,
    scoring_profile=None,
    include_total_count=True
)

# Print the total count of documents matching the query
print('Total Documents Matching Query:', results.get_count())

# Print the results
for result in results:
    print(result["@search.score"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")
    print(f"Category: {result['Category']}")
    print(f"Tags: {', '.join(result['Tags'])}")
    print(f"Rating: {result['Rating']}")
    print("-" * 50)

Total Documents Matching Query: 2
1.6811352
Twin Dome Motel
Description: The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts.
Category: Boutique
Tags: pool, free wifi, concierge
Rating: 3.6
--------------------------------------------------
0.25426257
Secret Point Motel
Description: The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.
Category: Boutique
Tags: pool, air conditioning, concierge
Rating: 3.6
--------------------------------------------------


## Run a term query

In [69]:
# Run a text query (returns a BM25-scored result set)
results =  search_client.search(query_type='simple',
    search_text="what hotel has a good restaurant on site" ,
    select='HotelName,HotelId,Description',
    include_total_count=True)
    
for result in results:
    print(result["@search.score"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

2.1393623
Sublime Cliff Hotel
Description: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace.
1.9309065
Twin Dome Motel
Description: The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts.
1.5589908
Triple Landscape Hotel
Description: The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services.
0.7704947
Secret Point Motel
Description: The hotel is ideally located on the main commercial artery of the ci

## Run a semantic query

- NEED TO set **Semantic ranker**

In [71]:
# Runs a semantic query (runs a BM25-ranked query and promotes the most relevant matches to the top)
results =  search_client.search(query_type='semantic', semantic_configuration_name='my-semantic-config',
    search_text="what hotel has a good restaurant on site", 
    select='HotelName,Description,Category', query_caption='extractive')

for result in results:
    print(result["@search.reranker_score"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

1.8807636499404907
Triple Landscape Hotel
Description: The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services.
Caption: Triple Landscape Hotel. Resort and Spa. The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services..

1.527956485748291
Sublime Cliff Hotel
Description: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace.
Caption: <em>Sublime Cliff Hotel.</em> Boutique.<em> Sublime Cliff Hotel</em> is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short 

## Return semantic answers

In [72]:
# Run a semantic query that returns semantic answers  
results =  search_client.search(query_type='semantic', semantic_configuration_name='my-semantic-config',
 search_text="what hotel is in a historic building",
 select='HotelName,Description,Category', query_caption='extractive', query_answer="extractive",)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(result["@search.reranker_score"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Semantic Answer: <em>Sublime Cliff Hotel.</em> Boutique.<em> Sublime Cliff Hotel</em> is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace..
Semantic Answer Score: 0.74072265625

2.4754135608673096
Twin Dome Motel
Description: The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts.
Caption: Twin Dome Motel. Boutique. The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique 

## Vector Serch

https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/basic-vector-workflow/azure-search-vector-python-sample.ipynb

## Clean up

If you're finished with this index, you can delete it by running the following lines. Deleting unnecessary indexes frees up space for stepping through more quickstarts and tutorials.

In [ ]:
# try:
#     result = index_client.delete_index(index_name)
#     print ('Index', index_name, 'Deleted')
# except Exception as ex:
#     print (ex)

Confirm the index deletion by running the following script that lists all of the indexes on your search service. If hotels-quickstart is not listed, you've successfully deleted the index and have completed this quickstart.

In [ ]:
# try:
#     result = index_client.get_index(index_name)
#     print (result)
# except Exception as ex:
#     print (ex)
